In [1]:
import pandas as pd
import os

os.chdir(r"C:\Users\meama\Documents\A5\AAI_Machine_Learning_for_NLP\Project2")

In [2]:
df=pd.read_csv(r"C:\Users\meama\Documents\A5\AAI_Machine_Learning_for_NLP\Project2\df_new_format_trad.csv")
df.head()

,name,link,description,average_score,number_votes,comment,score,email,phone_number,location,language,language_com2,description_trad
0,American Handyman in Paris,https://www.yelp.fr/biz/american-handyman-in-p...,Petits travaux à domicile Électricien Plombier...,5.0,6,When you are in the US and your problem is in ...,5.0,americanhandymaninparis.fr,06 12 32 04 39,no location,en,en,Small home jobs Electrician Plumber Appliance ...
1,American Handyman in Paris,https://www.yelp.fr/biz/american-handyman-in-p...,Petits travaux à domicile Électricien Plombier...,5.0,6,"If you are wondering if you should call him, I...",5.0,americanhandymaninparis.fr,06 12 32 04 39,no location,en,en,Small home jobs Electrician Plumber Appliance ...
2,American Handyman in Paris,https://www.yelp.fr/biz/american-handyman-in-p...,Petits travaux à domicile Électricien Plombier...,5.0,6,Have had multiple plumbing issues in my apartm...,5.0,americanhandymaninparis.fr,06 12 32 04 39,no location,en,en,Small home jobs Electrician Plumber Appliance ...
3,American Handyman in Paris,https://www.yelp.fr/biz/american-handyman-in-p...,Petits travaux à domicile Électricien Plombier...,5.0,6,Very efficient and quick! Maxim responded to m...,5.0,americanhandymaninparis.fr,06 12 32 04 39,no location,en,en,Small home jobs Electrician Plumber Appliance ...
4,American Handyman in Paris,https://www.yelp.fr/biz/american-handyman-in-p...,Petits travaux à domicile Électricien Plombier...,5.0,6,We had furniture delivered when we moved to Pa...,5.0,americanhandymaninparis.fr,06 12 32 04 39,no location,en,en,Small home jobs Electrician Plumber Appliance ...


In [4]:
df.comment.isna().sum()

1169

In [5]:
df.comment.fillna("", inplace=True)
df.comment=df.comment.astype(str)
df.comment.isna().sum(), df.comment.dtypes

(0, dtype('O'))

## T5 summary

In [6]:
import torch
print(torch.__version__)

2.1.2+cpu


## Pre-trained1

In [6]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the fine-tuned model
model_name = r"C:\Users\meama\Documents\A5\AAI_Machine_Learning_for_NLP\Project2\t5_abs_sum"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [15]:
def getSummary(review_text):
    if review_text==None :
        return review_text
    # Tokenize the text for T5 input
    input_ids = tokenizer(review_text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Generate the summary
    summary_ids = model.generate(input_ids['input_ids'], max_length=150, length_penalty=2.0, num_beams=4, early_stopping=True)

    # Convert the summary token ids to text
    summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary_text

In [ ]:
# Assuming your DataFrame is named 'df'
businesses = df['name'].unique()

# Initialize a dictionary to store summaries
summaries = {}

# Loop through each unique business
for business_name in businesses:
    # Filter comments for the current business
    business_comments = df[df['name'] == business_name]['comment'].tolist()

    # Check if there are comments for the business
    if not business_comments:
        summaries[business_name] = "No comments available"
        continue

    # Concatenate all comments into a single text
    combined_comments = " ".join(business_comments)

    # Generate a summary using the BERT model
    #summary = model(combined_comments, min_length=0, max_length=100)  
    summary = getSummary(combined_comments)
    summaries[business_name] = summary

# Store the summaries in your DataFrame
df['comment_summary'] = df['name'].map(summaries)

In [ ]:
df.to_csv(r"C:\Users\meama\Documents\A5\AAI_Machine_Learning_for_NLP\Project2\df_nf_summarized.csv", index=False)

## Pre-trained2

In [7]:
from transformers import AutoModelWithLMHead, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-summarize-news")
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-summarize-news")

def summarize(text, max_length=150):
  input_ids = tokenizer.encode(text, return_tensors="pt", add_special_tokens=True)

  generated_ids = model.generate(input_ids=input_ids, num_beams=2, max_length=max_length,  repetition_penalty=2.5, length_penalty=1.0, early_stopping=True)

  preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]

  return preds[0]


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
C:\Users\meama\anaconda3\envs\env1\lib\site-packages\transformers\models\auto\modeling_auto.py:1564: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [8]:
"""print(df['name'].unique().shape)
# Assuming your DataFrame is named 'df'
businesses = df['name'].unique()

# Initialize a dictionary to store summaries
summaries = {}

# Loop through each unique business
for business_name in businesses:
    # Filter comments for the current business
    business_comments = df[df['name'] == business_name]['comment'].tolist()

    # Check if there are comments for the business
    if not business_comments:
        summaries[business_name] = "No comments available"
        continue

    # Concatenate all comments into a single text
    combined_comments = " ".join(business_comments)

    # Generate a summary using the BERT model
    #summary = model(combined_comments, min_length=0, max_length=100)  
    summary = summarize(combined_comments)
    summaries[business_name] = summary

# Store the summaries in your DataFrame
df['comment_summary'] = df['name'].map(summaries)"""

'print(df[\'name\'].unique().shape)\n# Assuming your DataFrame is named \'df\'\nbusinesses = df[\'name\'].unique()\n\n# Initialize a dictionary to store summaries\nsummaries = {}\n\n# Loop through each unique business\nfor business_name in businesses:\n    # Filter comments for the current business\n    business_comments = df[df[\'name\'] == business_name][\'comment\'].tolist()\n\n    # Check if there are comments for the business\n    if not business_comments:\n        summaries[business_name] = "No comments available"\n        continue\n\n    # Concatenate all comments into a single text\n    combined_comments = " ".join(business_comments)\n\n    # Generate a summary using the BERT model\n    #summary = model(combined_comments, min_length=0, max_length=100)  \n    summary = summarize(combined_comments)\n    summaries[business_name] = summary\n\n# Store the summaries in your DataFrame\ndf[\'comment_summary\'] = df[\'name\'].map(summaries)'

In [9]:
"""def summarize(text, max_length=150):
    # Break the text into chunks
    chunk_size = 400  # Adjust as needed
    chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

    summaries = []
    for chunk in chunks:
        input_ids = tokenizer.encode(chunk, return_tensors="pt", add_special_tokens=True)
        generated_ids = model.generate(input_ids=input_ids, num_beams=2, max_length=max_length, repetition_penalty=2.5, length_penalty=1.0, early_stopping=True)
        chunk_summary = tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
        summaries.append(chunk_summary)

    return " ".join(summaries)

print(df['name'].unique().shape)
businesses = df['name'].unique()[:2]

summaries = {}
for i, business_name in enumerate(businesses):
    print(i, end='\r')
    business_comments = df[df['name'] == business_name]['comment'].tolist()
    if not business_comments:
        summaries[business_name] = "No comments available"
        continue

    combined_comments = " ".join(business_comments)
    summary = summarize(combined_comments)
    summaries[business_name] = summary

df['comment_summary'] = df['name'].map(summaries)"""

'def summarize(text, max_length=150):\n    # Break the text into chunks\n    chunk_size = 400  # Adjust as needed\n    chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]\n\n    summaries = []\n    for chunk in chunks:\n        input_ids = tokenizer.encode(chunk, return_tensors="pt", add_special_tokens=True)\n        generated_ids = model.generate(input_ids=input_ids, num_beams=2, max_length=max_length, repetition_penalty=2.5, length_penalty=1.0, early_stopping=True)\n        chunk_summary = tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)\n        summaries.append(chunk_summary)\n\n    return " ".join(summaries)\n\nprint(df[\'name\'].unique().shape)\nbusinesses = df[\'name\'].unique()[:2]\n\nsummaries = {}\nfor i, business_name in enumerate(businesses):\n    print(i, end=\'\r\')\n    business_comments = df[df[\'name\'] == business_name][\'comment\'].tolist()\n    if not business_comments:\n        summaries[business_n

In [ ]:
import concurrent.futures

def process_business(business_name):
    business_comments = df[df['name'] == business_name]['comment'].tolist()
    if not business_comments:
        return business_name, "No comments available"

    combined_comments = " ".join(business_comments)
    summary = summarize(combined_comments)
    return business_name, summary

# Use multi-threading for concurrent processing
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(process_business, business_name) for business_name in businesses]
    summaries = {business_name: summary for business_name, summary in [future.result() for future in futures]}

df['comment_summary'] = df['name'].map(summaries)


In [8]:
import concurrent.futures
from tqdm import tqdm

businesses = df['name'].unique()

def process_business(business_name):
    business_comments = df[df['name'] == business_name]['comment'].tolist()
    if not business_comments:
        return business_name, "No comments available"

    combined_comments = " ".join(business_comments)
    summary = summarize(combined_comments)
    return business_name, summary

# Use ThreadPoolExecutor for parallel processing
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Prepare futures and wrap them with tqdm for progress display
    futures = [executor.submit(process_business, business_name) for business_name in businesses]
    results = [future.result() for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures))]

# Extracting summaries from results
summaries = dict(results)

# Mapping summaries to the dataframe
df['comment_summary'] = df['name'].map(summaries)


Token indices sequence length is longer than the specified maximum sequence length for this model (532 > 512). Running this sequence through the model will result in indexing errors
100%|██████████| 1999/1999 [3:17:08<00:00,  5.92s/it]  


In [17]:
df['comment_summary']

0       You can trust them, perfect English is spoken,...
1       You can trust them, perfect English is spoken,...
2       You can trust them, perfect English is spoken,...
3       You can trust them, perfect English is spoken,...
4       You can trust them, perfect English is spoken,...
                              ...                        
6689                                                  NaN
6690                                                  NaN
6691                                                  NaN
6692                                                  NaN
6693                                                  NaN
Name: comment_summary, Length: 6694, dtype: object

In [ ]:
summaries

In [9]:
df.to_csv(r"C:\Users\meama\Documents\A5\AAI_Machine_Learning_for_NLP\Project2\df_new_format_abs_summarized2.csv", index=False)

## Train T5

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

# Load a pre-trained T5 model and tokenizer
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Prepare your data (you'll need to format your data as input-output pairs)
input_texts = ["input text 1", "input text 2"]
target_texts = ["target text 1", "target text 2"]

# Encode the inputs and targets
input_encodings = tokenizer(input_texts, padding=True, return_tensors="pt")
target_encodings = tokenizer(target_texts, padding=True, return_tensors="pt")

# Define a training loop (this is highly simplified)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(input_ids=input_encodings['input_ids'], 
                    labels=target_encodings['input_ids'])
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch} loss: {loss.item()}")

# Save the fine-tuned model
model.save_pretrained('./my_finetuned_t5')
